In [3]:
import os, sys
import re
import numpy as np
import imageio as im
import pandas as pd
import cv2
from skimage.morphology import skeletonize
from skimage.util import invert
from IPython.display import Image
import matplotlib.pyplot as plt
from cv2 import imwrite
from cv_algorithms import grassfire
from cv_algorithms import thinning as ta
from cv_algorithms.morphology import difference_of_gaussian
from fil_finder import FilFinder2D
from astropy.io import fits
import astropy.units as u
from IPython.core.display import HTML
cell_path = "individual_cells/"

In [12]:
def prune(skel,mask,b_thresh=40,sk_thresh=10):
    fil=FilFinder2D(skel,mask=mask)
    fil.preprocess_image(skip_flatten=True)
    fil.medskel()
    fil.analyze_skeletons(branch_thresh=b_thresh*u.pix, skel_thresh=sk_thresh*u.pix, prune_criteria='length')
    return fil.skeleton_longpath>0
def fskel(mask, b_thresh=40, sk_thresh=20):
    fil=FilFinder2D(mask,mask=mask)
    fil.preprocess_image(skip_flatten=True)
    fil.create_mask(use_existing_mask=True)
    fil.medskel(verbose=False)
    unpruned_skel = fil.skeleton
    fil.analyze_skeletons(branch_thresh=b_thresh*u.pix, skel_thresh=sk_thresh*u.pix, prune_criteria='length',max_prune_iter=20)
    skel = fil.skeleton_longpath
    return unpruned_skel,skel

In [13]:
all_the_cells = os.listdir(cell_path)

# Create a list that records the timepoints for every mask. 
time_list = []
# List of masks with associated timepoints and names
maskl = []
#list of skeletons made by thinning with associated timepoints
thinl = []
#list of skeletons made by guo_hall thinning algorithm with assocciated timepoints
ghtl = []
#list of skeletons made zhang-suen thinning algorithm with associated timepoints
zstl=[]
#list of skeletons made using the grassfire algorithm with associated timepoints
grasl=[]
#list of skeletons made using the difference of gaussians algorithm with associated timepoints
dogl=[]
#list of skeletons made using the filfinder algorithm with associated timepoints
fill=[]
for item in os.listdir('algorithm_comparison'):
    if item[-3]=='p' and item[-2]=='n' and item[-1]=='g':
        os.remove('algorithm_comparison'+'/'+item)
for cell in all_the_cells:
    time = re.findall(r'\d+', cell)
    if time != []:
        #overwrite old skeletons in individual_cells
        '''
        for item in os.listdir(cell_path+cell+'\skeletons'):
            if item[-3]=='p' and item[-2]=='n' and item[-1]=='g':
                os.remove(cell_path+cell+'\skeletons'+'/'+item)
        '''
        time = time[0]
        time_list.append(int(time))
        im_path = cell_path + cell + '\masks'
        for mask_name in os.listdir(im_path):
            mask=im.imread(im_path+'/'+mask_name)[:,:,0]>0 #the cells mask (boolean array)
            #unpruned_thin=skeletonize(mask) #the masks skeleton from skimage (boolean array)
            #thin = prune(unpruned_thin*255,mask)
            #thin = invert (thin + invert(mask))
            #unpruned_ght = ta.guo_hall(mask.astype(np.uint8))>0 #the masks skeleton from the guo_hall algorithm (boolean array)
            #ght=prune(unpruned_ght*255,mask)
            #ght = invert (ght + invert(mask))
            #unpruned_zst = ta.zhang_suen(mask.astype(np.uint8))>0 # the masks skeleton from the zhang_shuen algorithm (boolean array)
            #zst=prune(unpruned_zst*255,mask)
            #zst = invert(zst + invert(mask))
            unpruned_fil, fil = fskel(mask*255)
            fil = invert(fil+invert(mask))
            maskl.append([mask,time,int(mask_name[5:6])])
            #thinl.append([thin,time,int(mask_name[5:6])])
            #ghtl.append([ght,time,int(mask_name[5:6])])
            #zstl.append([zst,time,int(mask_name[5:6])])
            fill.append([fil,time,int(mask_name[5:6])])
            imwrite('algorithm_comparison'+'/'+str(time)+'_'+mask_name[5:6]+'_pruned'+'.png',255*fil)
            imwrite('algorithm_comparison'+'/'+str(time)+'_'+mask_name[5:6]+'_unpruned'+'.png',255*unpruned_fil)
            #imwrite('algorithm_comparison'+'/'+str(time)+'_'+mask_name[5:6]+'_guo_hall'+'.png',255*ght)
            #imwrite('algorithm_comparison'+'/'+str(time)+'_'+mask_name[5:6]+'_zhang_suen'+'.png',255*zst)

c:\users\sammu\appdata\local\programs\python\python39\lib\site-packages\fil_finder\filfinder2D.py:142: UserWarning: No beam width given. Using 0 pixels.
  warnings.warn("No beam width given. Using 0 pixels.")
c:\users\sammu\appdata\local\programs\python\python39\lib\site-packages\fil_finder\filfinder2D.py:296: UserWarning: Using inputted mask. Skipping creation of anew mask.
  warnings.warn("Using inputted mask. Skipping creation of a"
c:\users\sammu\appdata\local\programs\python\python39\lib\site-packages\fil_finder\filament.py:326: UserWarning: Graph pruning reached max iterations.
  warnings.warn("Graph pruning reached max iterations.")


In [14]:
pruned = []
unpruned = []
times = []
idl = []
for mask in maskl:
    pruned.append('algorithm_comparison'+'/'+mask[1]+'_'+str(mask[2])+'_pruned.png')
    unpruned.append('algorithm_comparison'+'/'+mask[1]+'_'+str(mask[2])+'_unpruned.png')
    times.append(mask[1])
    idl.append(mask[2])

In [15]:
skeldf=pd.DataFrame()
skeldf['Pruned']=pruned
skeldf['Unpruned']=unpruned
skeldf['Time'] = times
skeldf['ID'] = idl
def skel_formatter(path):
    return '<img src="'+ path + '" width="256" >'

In [16]:
imagehtml = skeldf.to_html(escape=False,formatters=dict(Pruned=skel_formatter,Unpruned=skel_formatter))
HTML (imagehtml)

In [220]:
def bdr_not_reached_nw(submask):
    n = submask.shape[0]-1
    if submask[0,0]==False:
        submask[0,0]=True
    if submask[0,n]==False:
        submask[0,n]=True
    if submask[n,n]==False:
        submask[n,n]=True
    if submask[n,0]==False:
        submask[n,0]=True
    if np.all(submask)==True:
        return True

In [264]:
a=5
b=45
d=1
submask= mask[a-d:a+1,b-d:b+1]
no_bdr = bdr_not_reached_nw(submask)
m = max(a,b)
while no_bdr:
    d= d+1
    submask= mask[a-d:a+1,b-d:b+1]
    no_bdr = bdr_not_reached_nw(submask)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [266]:
a=5
b=45
d=1
submask= mask[a-d:a+1,b-d:b+1]
no_bdr = bdr_not_reached_nw(submask)

In [105]:
def decrease_by_one(gras):
    for i in range (0,np.shape(gras)[0]):
        for j in range(0,np.shape(gras)[1]):
            if gras[i,j]>1:
                gras[i,j]=gras[i,j]-1
            elif gras[i,j]=1:
                
    return gras

In [106]:
while np.max(gras)>0:
    gras = decrease_by_one(gras)